In [1]:
import faiss
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import HuggingFacePipeline

INDEX_NAME = "trainual.index"
STORE_NAME = "trainual.pkl"

# To Download LLAMA: https://github.com/facebookresearch/llama/issues/374#issuecomment-1643228958
model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
#     max_length=3773,
#     temperature=0,
#     top_p=0.95,
#     repetition_penalty=1.15,
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# Load the LangChain.
index = faiss.read_index(INDEX_NAME)

with open(STORE_NAME, "rb") as f:
    store = pickle.load(f)

store.index = index
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=local_llm, retriever=store.as_retriever(),
    return_source_documents=True
)

/home/bruno/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bruno/.local/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.08s/it]


In [3]:
input_ = input()
print(f"Question: {input_}")
result = chain({"question": input_})
print(f"Answer: {result['answer']}")
print(f"Sources: {result['sources']}")

Question: What are the steps to add PTO to Clarizen?
Answer:  The steps to add PTO to Clarizen are as follows:

1. Login to Clarizen
2. Look for your username at the top right corner
3. Pull down on your username and look for “My Details” link
4. Now you have to extend the details page by clicking on the “More info” link in the middle of the page
5. Look for the section called “Personal Details” and click on the edit link under Calendar
6. Add an exception for all the hours/days you are on PTO (the “Name” is any description, like “Vacation”)

The steps are important because they help Clarizen block out your PTO hours and adjust your resource load, tasks, and other data reporting things that BioTeam uses to run reports and charts.
Sources: 
